# Pylops - stacking with mixed inputs

### Author: M.Ravasi

In this notebook we consider stacking operators and pass mixed inputs (pylops operators, numpy matrices, scipy sparse matrices)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

import pylops

from scipy.sparse import csr_matrix, vstack, rand, bsr_matrix, eye
from scipy.linalg import lstsq, solve, pinv
from scipy.sparse.linalg import LinearOperator as spLinearOperator
from scipy.sparse.linalg import cg, lsqr

from pylops.utils                      import dottest
from pylops.basicoperators             import *

# Concatenation of linear operators

## Vertical stacking of operators

We now want to vertically stack 2 operators, one is an operator and one is a matrix

$$  \mathbf{D} =
    \begin{bmatrix}
    \mathbf{D_2}    \\
    \mathbf{M}    
    \end{bmatrix}$$

In [2]:
D2op = SecondDerivative(51, dtype='float64')
M = np.arange(51*51).reshape(51, 51)
Mop = MatrixMult(M,  dtype='float64')

Vstackop = VStack([D2op, M])
Vstackop1 = VStack([D2op, Mop])
dottest(Vstackop, 2*51, 51, verb=True)
dottest(Vstackop1, 2*51, 51, verb=True)

x = np.ones(51)
assert np.allclose(Vstackop * x, Vstackop1 * x)

Dot test passed, v^H(Opu)=115968.16803426258 - u^H(Op^Hv)=115968.16803426259
Dot test passed, v^H(Opu)=-20107.190953501795 - u^H(Op^Hv)=-20107.190953501795


and three operators

$$  \mathbf{D} =
    \begin{bmatrix}
    \mathbf{D}    \\
    0.5*\mathbf{M}    \\
    2*\mathbf{D_2}    
    \end{bmatrix}$$

In [3]:
D2op = SecondDerivative(51, dtype='float64')
Dop = FirstDerivative(51, dtype='float64')
M = eye(51)
Mop = MatrixMult(M,  dtype='float64')

Vstackop = VStack([D2op, 0.5*M, Dop])
Vstackop1 = VStack([D2op, 0.5*Mop, Dop])
dottest(Vstackop, 3*51, 51, verb=True)
dottest(Vstackop1, 3*51, 51, verb=True)

x = np.ones(51)
assert np.allclose(Vstackop * x, Vstackop1 * x)

Dot test passed, v^H(Opu)=-14.572131440491866 - u^H(Op^Hv)=-14.572131440491868
Dot test passed, v^H(Opu)=-26.976705315124782 - u^H(Op^Hv)=-26.97670531512478


And with complex numbers and scipy sparse

In [4]:
D2op = SecondDerivative(51, dtype='complex128')
Dop = FirstDerivative(51, dtype='complex128')
M = 1j*np.arange(51**2).reshape(51, 51)
Ms = 1j*eye(51)
Mop = MatrixMult(M,  dtype='complex128')
Msop = MatrixMult(Ms,  dtype='complex128')

Vstackop = VStack([D2op, 0.5*M, Ms], dtype='complex128')
Vstackop1 = VStack([D2op, 0.5*Mop, Msop], dtype='complex128')
dottest(Vstackop, 3*51, 51, atol=1e-10, complexflag=3, verb=True)
dottest(Vstackop1, 3*51, 51, atol=1e-10, complexflag=3, verb=True)

x = 1j*np.ones(51)
assert np.allclose(Vstackop * x, Vstackop1 * x)

Dot test passed, v^H(Opu)=(-76156.9362833265+91670.44018600257j) - u^H(Op^Hv)=(-76156.9362833265+91670.44018600261j)
Dot test passed, v^H(Opu)=(-144279.37066953833+109592.01278453437j) - u^H(Op^Hv)=(-144279.3706695383+109592.01278453435j)


## Horizontal stacking of operators

We now want to horizontally stack various operators

$$  \mathbf{D} =
    \begin{bmatrix}
    \mathbf{D_2}  \quad 0.5*\mathbf{M}   
    \end{bmatrix}$$

In [5]:
D2op = SecondDerivative(51, dtype='float64')
M = np.arange(51*61).reshape(51, 61)
Mop = MatrixMult(M,  dtype='float64')

Hstackop = HStack([D2op, M])
Hstackop1 = HStack([D2op, Mop])
dottest(Hstackop, 51, 51+61, verb=True)
dottest(Hstackop1, 51, 51+61, verb=True)

x = np.ones(51+61)
print(np.allclose(Hstackop * x, Hstackop1 * x))

Dot test passed, v^H(Opu)=-298650.1790791495 - u^H(Op^Hv)=-298650.17907914944
Dot test passed, v^H(Opu)=-40769.68730627748 - u^H(Op^Hv)=-40769.687306277476
True


And with complex numbers and scipy sparse

In [6]:
D2op = SecondDerivative(51, dtype='complex128')
Dop = FirstDerivative(51, dtype='complex128')
M = 1j*np.arange(51**2).reshape(51, 51)
Ms = 1j*eye(51)
Mop = MatrixMult(M,  dtype='complex128')
Msop = MatrixMult(Ms,  dtype='complex128')

Hstackop = HStack([D2op, 0.5*M, Ms], dtype='complex128')
Hstackop1 = HStack([D2op, 0.5*Mop, Msop], dtype='complex128')
dottest(Hstackop, 51, 3*51, atol=1e-10, complexflag=3, verb=True)
dottest(Hstackop1, 51, 3*51, atol=1e-10, complexflag=3, verb=True)

x = 1j*np.ones(3*51)
print(np.allclose(Hstackop * x, Hstackop1 * x))

Dot test passed, v^H(Opu)=(-3296.987326525763-19073.701405018368j) - u^H(Op^Hv)=(-3296.9873265257957-19073.70140501837j)
Dot test passed, v^H(Opu)=(26562.576948487356-44862.816811394536j) - u^H(Op^Hv)=(26562.57694848734-44862.81681139454j)
True


## Blockdiagonal matrix

We now want to create a block diagonal matrix

$$  \mathbf{D} =
    \begin{bmatrix}
    \mathbf{D_2}  \quad \mathbf{0}       \quad  \mathbf{0}  \\
    \mathbf{0}    \quad 0.5*\mathbf{M} \quad  \mathbf{0}  \\
    \mathbf{0}    \quad \mathbf{0}       \quad  -1*\mathbf{M_s}
    \end{bmatrix}$$

In [7]:
Bop = BlockDiag([D2op, 0.5*Mop, -1*Msop], dtype='complex128')
Bop1 = BlockDiag([D2op, 0.5*Mop, -1*Msop], dtype='complex128')
dottest(Bop, 3*51, 3*51, complexflag=3, verb=True)
dottest(Bop1, 3*51, 3*51, complexflag=3, verb=True)

x = 1j*np.ones(3*51)
assert np.allclose(Bop * x, Bop1 * x)

Dot test passed, v^H(Opu)=(16179.55658666357+46807.64479352242j) - u^H(Op^Hv)=(16179.556586663566+46807.64479352242j)
Dot test passed, v^H(Opu)=(14490.601692136654+46680.868970708216j) - u^H(Op^Hv)=(14490.601692136646+46680.86897070826j)


## Block matrix

In [8]:
bl1 = Block([[MatrixMult(np.ones((5,5))), -2*Identity(5)],
            [np.zeros((5,5)),             Diagonal(-2*np.ones(5))],
            [eye(5),                      Diagonal(-2*np.ones(5), dtype=np.float64)]])

In [9]:
bl = Block([[MatrixMult(np.ones((5,5))), -2*Identity(5)],
            [Zero(5),                    Diagonal(-2*np.ones(5))],
            [Identity(5),                Diagonal(-2*np.ones(5), dtype=np.float64)]])
bl1 = Block([[MatrixMult(np.ones((5,5))), -2*Identity(5)],
            [np.zeros((5,5)),             Diagonal(-2*np.ones(5))],
            [eye(5),                      Diagonal(-2*np.ones(5), dtype=np.float64)]])
dottest(bl, 15, 10,  verb=True)
dottest(bl1, 15, 10, verb=True)

x = np.ones(10)
assert np.allclose(bl * x, bl1 * x)

Dot test passed, v^H(Opu)=-0.3333290152414949 - u^H(Op^Hv)=-0.33332901524149383
Dot test passed, v^H(Opu)=4.162734352056086 - u^H(Op^Hv)=4.162734352056086
